# Create subjects from a list

Dict of subject names with ORCiD URLs

In [46]:
from d1_client.cnclient import CoordinatingNodeClient
from d1_common.types.dataoneTypes import Subject, person
from os import environ

AUTH_TOKEN = {
    'prod': 'TOKEN',
    'stage': 'TOKEN',
}
CN_URL = {
    'prod': 'https://cn.dataone.org/cn',
    'stage': 'https://cn-stage.test.dataone.org/cn',
}
CN = 'stage'
environ['D1_AUTH_TOKEN'] = AUTH_TOKEN[CN]
auth_token = environ.get('D1_AUTH_TOKEN')

# Set the token in the request header
options: dict = {"headers": {"Authorization": "Bearer " + auth_token}}
# Create the Member Node Client
client: CoordinatingNodeClient = CoordinatingNodeClient(CN_URL[CN], **options)

## Test user

In [26]:
id_dict = {'http://orcid.org/0000-0001-5828-6070': ['Ian Nesbitt', 'nesbitt@nceas.ucsb.edu']}
for sid in id_dict:
    name = id_dict[sid][0]
    name_list = name.split(' ')
    given_name = id_dict[sid][0].split()[:-1]
    family_name = id_dict[sid][0].split()[-1]
    s = Subject(sid)
    p = person()
    p.subject = s
    p.givenName = given_name
    p.familyName = family_name
    p.email = 'nesbitt@nceas.ucsb.edu'
    try:
        client.registerAccount(p)
    except Exception as e:
        err_n = str(e).split('\n')[0]
        err_c = str(e).split('\n')[1]
        err_d = str(e).split('\n')[3]
        print('Error processing %s (%s)\n%s\n%s\n%s' % (name, sid, err_n, err_c, err_d))

Error processing Ian Nesbitt (http://orcid.org/0000-0001-5828-6070)
name: IdentifierNotUnique
errorCode: 409
description: Entry uid=http://orcid.org/0000-0001-5828-6070,dc=dataone,dc=org already exists


In [44]:
id_list = {
    'http://orcid.org/0000-0002-6014-8942': ['Timothy Pasch', 'timothy.pasch@und.edu'],
}

## Create users from list

In [31]:
for orcid in id_list:
    name = id_list[orcid][0]
    nameL = name.split(' ')
    givenNameL = id_list[orcid][0].split()[:-1]
    familyName = id_list[orcid][0].split()[-1]
    email = id_list[orcid][1]
    s = Subject(orcid)
    p = person()
    p.subject = s
    p.givenName = givenNameL
    p.familyName = familyName
    p.email = email
    p.verified = True
    
    n = 1
    listLen = len(id_list[orcid])
    while n < listLen:
        p.subject = id_list[orcid][n]
        n += 1
    print('%s %s %s' % (p.content()[0].value(), p.content()[1], p.content()[2]))
    if True:
        try:
            client.registerAccount(p)
        except Exception as e:
            err_n = str(e).split('\n')[0]
            err_c = str(e).split('\n')[1]
            err_d = str(e).split('\n')[3]
            print('Error processing %s (%s)\n%s\n%s\n%s' % (name, orcid, err_n, err_c, err_d))

http://orcid.org/0000-0002-6014-8942 ['Timothy'] Pasch
Error processing Timothy Pasch (http://orcid.org/0000-0002-6014-8942)
name: IdentifierNotUnique
errorCode: 409
description: Entry uid=http://orcid.org/0000-0002-6014-8942,dc=dataone,dc=org already exists
http://orcid.org/0000-0001-6699-2020 ['Alexander'] Brand
Error processing Alexander Brand (http://orcid.org/0000-0001-6699-2020)
name: IdentifierNotUnique
errorCode: 409
description: Entry uid=http://orcid.org/0000-0001-6699-2020,dc=dataone,dc=org already exists
http://orcid.org/0000-0002-2137-4103 ['Olaf'] Kuhlke
Error processing Olaf Kuhlke (http://orcid.org/0000-0002-2137-4103)
name: IdentifierNotUnique
errorCode: 409
description: Entry uid=http://orcid.org/0000-0002-2137-4103,dc=dataone,dc=org already exists
http://orcid.org/0000-0003-0910-5688 ['Emma'] Verstraete
Error processing Emma Verstraete (http://orcid.org/0000-0003-0910-5688)
name: IdentifierNotUnique
errorCode: 409
description: Entry uid=http://orcid.org/0000-0003-091

In [33]:
srch = client.getSubjectInfo(orcid)
srch

In [35]:
srch.content()[0]